# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings.csv',low_memory=False)

In [5]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,genres,popularity,budget,original_language,id,revenue,runtime,original_title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,30000000,en,862,373554033.0,81.0,Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,65000000,en,8844,262797249.0,104.0,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,0,en,15602,0.0,101.0,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,16000000,en,31357,81452156.0,127.0,Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",8.387519,0,en,11862,76578911.0,106.0,Father of the Bride Part II


In [7]:
#Tirando budgets diferente de 0 (budget ainda como string)
df=df[df.budget != "0"]
df=df.drop([19730, 29503, 35587])

In [8]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

genres                object
popularity            object
budget               float64
original_language     object
id                   float64
revenue              float64
runtime              float64
original_title        object
dtype: object

In [9]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df.id=df.id.astype('int')

In [10]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
MediaRating=imdb.drop("userId",1,inplace=True)
MediaRating=imdb.drop("timestamp",1,inplace=True)
imdb.columns

Index(['movieId', 'rating'], dtype='object')

In [11]:
imdb.tail()

,movieId,rating
18234777,176267,4.0
9448563,176269,3.5
83813,176271,5.0
25735653,176273,1.0
17181377,176275,3.0


In [12]:
rating=pd.Series(imdb.groupby(['movieId'])['rating'].mean())

In [13]:
rating=rating.reset_index()
rating = rating.rename(columns={'movieId': 'id'})
rating=rating.set_index('id')

In [14]:
df=df.set_index("id")

In [15]:
dados_filtrados=pd.merge(df, rating, on='id', how='inner')
dados_filtrados

,genres,popularity,budget,original_language,revenue,runtime,original_title,rating
id,,,,,,,,
5,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",9.026586,4000000.0,en,4.300000e+06,98.0,Four Rooms,3.079565
11,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",42.149697,11000000.0,en,7.753980e+08,121.0,Star Wars,3.660591
12,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",25.497794,94000000.0,en,9.403355e+08,100.0,Finding Nemo,2.672179
13,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",48.307194,55000000.0,en,6.779454e+08,142.0,Forrest Gump,3.326442
14,"[{'id': 18, 'name': 'Drama'}]",20.726578,15000000.0,en,3.562966e+08,122.0,American Beauty,3.430807
15,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",15.811921,839727.0,en,2.321767e+07,119.0,Citizen Kane,2.727040
16,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",10.684806,12800000.0,en,4.003188e+07,140.0,Dancer in the Dark,3.795511
18,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",24.30526,90000000.0,en,2.639202e+08,126.0,The Fifth Element,3.409031
19,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",14.487867,92620000.0,de,6.504220e+05,153.0,Metropolis,2.633983


In [16]:
dados_filtrados.to_csv("dataframe_filtrado.csv")